In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from missingno import matrix as mm
from matplotlib import pyplot as plt

In [2]:
data = pd.read_csv('../../data/amsc.csv')
data.head(5)

,RT,SERIALNO,SPORDER,PUMA,ST,ADJUST,PWGTP,AGEP,CIT,COW,...,pwgtp71,pwgtp72,pwgtp73,pwgtp74,pwgtp75,pwgtp76,pwgtp77,pwgtp78,pwgtp79,pwgtp80
0,P,186,1,700,16,1015675,89,43,1,7.0,...,82,27,92,150,28,78,25,99,159,129
1,P,186,2,700,16,1015675,92,42,1,4.0,...,84,26,90,159,30,87,27,98,167,131
2,P,186,3,700,16,1015675,107,16,1,1.0,...,90,28,92,177,33,105,30,104,206,156
3,P,186,4,700,16,1015675,91,14,1,NaN,...,81,28,94,164,29,88,27,104,156,138
4,P,306,1,700,16,1015675,309,29,1,5.0,...,294,400,80,489,340,491,612,282,462,259


In [3]:
# info
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14931 entries, 0 to 14930
Columns: 239 entries, RT to pwgtp80
dtypes: float64(74), int64(162), object(3)
memory usage: 27.2+ MB


In [7]:
mask = data.isna()
mask.sum().sort_values(ascending=False)

GCM       14827
GCR       14724
SFR       14702
SFN       14702
DECADE    14122
          ...  
FWAGP         0
FSSP          0
FSSIP         0
FSEXP         0
RT            0
Length: 239, dtype: int64